In [ ]:
import warnings
from IPython.display import HTML, display

def set_css():
    display(HTML("<style> pre { white-space: pre-wrap; } </style>"))

get_ipython().events.register("pre_run_cell", set_css)
warnings.filterwarnings("ignore")

## 安裝套件

因為 Colab 免費版的 GPU 只有 16 GB 可以用，讀取 LLM 勢必需要進行量化 (Quantization)

因此除了 `transformers` 以外，安裝 `accelerate` 與 `bitsandbytes` 都是必要的

In [ ]:
%pip install transformers accelerate bitsandbytes sentencepiece -U

## 讀取模型

這邊讀取 Llama-2 7B Chat 模型

因為原本的 Llama-2 需要權限並設定 SSH 公鑰才能直接下載

因此這邊使用 TheBloke 大神上傳的 FP16 版本做示範

[TheBloke](https://huggingface.co/TheBloke) 是 HuggingFace 社群上非常活躍的用戶

經常幫各個模型做 Quantization 並重新上傳

In [ ]:
from transformers import BitsAndBytesConfig
from transformers import LlamaForCausalLM as ModelCls
from transformers import LlamaTokenizer as TkCls

model_path = "TheBloke/Llama-2-7b-chat-fp16"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # load_in_4bit=True,
)
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
)
tk: TkCls = TkCls.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt_template = "### USER: {}\n### ASSISTANT:"
prompt = prompt_template.format("什麼是大型語言模型？")
inputs = tk(prompt, return_tensors="pt").to("cuda")
print(inputs)

{'input_ids': tensor([[    1,   835,  3148,  1001, 29901, 29871,   231,   190,   131,   236,
           189,   191, 30392, 30257, 30883, 30968, 31243, 31382, 30883, 30882,
            13,  2277, 29937,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]], device='cuda:0')}


## .generate() 基本用法

In [ ]:
output = model.generate(**inputs, max_new_tokens=32)
print(tk.batch_decode(output)[0])

<s> ### USER: 什麼是大型語言模型？
### ASSISTANT: 大型語言模型（Large Language Model，LLM）是一種基於深度學習的


## TextStreamer

使用 TextStreamer 可以將文字用串流的方式一個一個印出來

In [ ]:
from transformers import TextStreamer

output = model.generate(
    **inputs,
    max_new_tokens=32,
    streamer=TextStreamer(tk)
)

<s> ### USER: 什麼是大型語言模型？
### ASSISTANT: 大型語言模型（Large Language Model，LLM）是一種基於深度學習的


## Stop Words

透過 StoppingCriteria 可以讓模型遇到指定文字時停止生成

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopWords(StoppingCriteria):
    def __init__(self, tk: TkCls, stop_words: list[str]):
        self.tk = tk
        self.stop_tokens = stop_words

    def __call__(self, input_ids, *_) -> bool:
        s = self.tk.batch_decode(input_ids)[0]
        for t in self.stop_tokens:
            if s.endswith(t):
                return True
        return False

sw = StopWords(tk, ["。", "！", "？"])
scl = StoppingCriteriaList([sw])

In [ ]:
output = model.generate(
    **inputs,
    max_new_tokens=2048,
    streamer=TextStreamer(tk),
    stopping_criteria=scl,
)

<s> ### USER: 什麼是大型語言模型？
### ASSISTANT: 大型語言模型（Large Language Model，LLM）是一種基於深度學習的語言模型，它可以從大量的文本數據中學習語言的構造和推理能力，以提供更好的語言處理和生成能力。


## Autoregressive Decoding

拆解自迴歸解碼的步驟

In [ ]:
import torch

curr_input_ids = inputs["input_ids"]
pkv = None

output_tokens = list()
for i in range(16):
    with torch.no_grad():
        outputs = model(curr_input_ids, past_key_values=pkv)
    next_token = outputs.logits.argmax(-1)[:, -1:]
    token_id = next_token.detach().cpu().numpy()[0][0]

    if token_id == tk.eos_token_id:
        break

    output_tokens.append(token_id)
    curr_input_ids = next_token
    pkv = outputs.past_key_values

    probs = torch.softmax(outputs.logits, -1)
    token_prob = probs[-1][-1][token_id].item()

print(tk.decode(output_tokens))

大型語言模型（Large Language Model，LLM）
